*TP2 : INDEXATION D’IMAGES*

*Réalisé par : ESSO Dissirama - NIKUE AMASSAH Djahlin Hervé*

*P23/SIM*

*Année scolaire 2019-2020*

In [1]:
# Importation de mon drive dans colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Installation de opencv 3.3.0.10
pip install opencv-python==3.3.0.10 opencv_contrib-python==3.3.0.10

     |████████████████████████████████| 15.5MB 294kB/s 
     |████████████████████████████████| 21.4MB 54.1MB/s 
ERROR: dopamine-rl 1.0.5 has requirement opencv-python>=3.4.1.15, but you'll have opencv-python 3.3.0.10 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [0]:
# copie de la base d'image dans colab
cp -R /content/drive/My\ Drive/image.orig /

In [0]:
# Importation des bibliothèques nécessaires
import numpy as np
import pickle
import cv2
import os

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

# Définition du chemain d'accès à la base d'images
chemain = "/image.orig"

# Nombre de cluster pour créer le sac de mots visuels avec kmeans fonction
n_cluster = 300
# Tableau pour stocker les descripteurs à extraire des images
descripteur_table = []

# Dictionnaire contenant le nom des images et les descripteurs asociés à chaque image
sift_vectors = {}

# Tableau contenant les descripteur des images ordonnés par classe
descripteur = []


*Première partie  : Conception d’un moteur de recherche d’images simple (sans
                            structuration des index*)

In [0]:

# Fonction de création du sac de mots visuels avec kmeans avec 2 parametres :
# K = nombre de cluster et descriptor_list = liste des descripteurs sift des images
# La fonction retourne le modèle kmeans créé à partir des descripteurs de chaque image
def bag_of_word_with_kmeans(k, descriptor_list):
    # Initialisation du Kmeans
    kmeans = KMeans(n_clusters=k, n_init=10)
    # Entraînement du kmeans sur la liste de descripteurs pour créer le modèle kmeans
    kmeans.fit(descriptor_list)

    # Retourner le modèle kmeans formé
    return kmeans


# function de construction d'histogramme d'une image
# Prend 2 paramètres : le model kmeans obtenu (sac de mot visuels) et les descripteurs correspondant à une image
# Retourne l'histogramme normalisé d'une image
def build_image_histogram(clustering_model, descriptors):
    # Initialisation de la taille de l'histogramme de l'image avec le nombre de cluster
    hist = np.zeros(n_cluster)
    # Calcule du nombre total de descripteur pour une image afin de l'utiliser pour normaliser l'histogramme
    n_desc = np.size(descriptors)
    # Parcours des descripteurs d'une image afin de calculer son histogramme
    for desc in descriptors:
        # Récupération de l'indice du cluster auquel le descripteur est plus proche
        idx = clustering_model.predict([desc])
        # Construction de l'histogramme de l'image avec le cluster prédict et le pourcentage du nombre de descripteurs
        # associé à ce cluster. On ajoute à l'indice idx (cluster plus proche) du tableau de l'histogramme (hist) le pourcentage du nombre de descripteurs
        # proche du cluster prédict par le modèle kmeans sur chaque descripteur de l'image.
        hist[idx] += 1/n_desc

    # retourner le tableau contenant l'histogramme de l'image
    return hist

# function de construction d'histogramme de toutes les imagees
# Prend 2 paramètres : le model kmeans obtenu (sac de mot visuels) et la liste des descripteurs ordonnée par classe
# Retourne un tableau contenant les histogrammes de toutes les images
def build_all_images_histogram(clustering_model, imgs_descriptors_list):
    # Initialisation du tableau des histogrammes
    histograms = []
    # Parcourir la liste des descripteurs des images pour récupérer les descripteurs de chaque image
    # pour calculer l'histogramme de chaque image. Ensuite on ajoute l'histogramme calculé dans le
    # tableau des histogrammes.
    for des in imgs_descriptors_list:
        # Calcul l'histogramme d'une image
        hist = build_image_histogram(clustering_model, des)
        # Ajout de l'histogramme d'une image dans le tableau des histogrammes
        histograms.append(hist)

    # Retourner le tableau contenant tous les histogrammes des images
    return histograms



# Fonction indexation : Fonction pour extraire les descripteurs des images de la base et créer le sac de mots visuels
# en utilisant kmeans.
# La fonction retourne le modèle kmeans formé sur les images et le tableau des histogrammes des images.
def indexation(chemain):
    # Parcourir les images dans la base d'images pour extraire les descripteurs des images
    for path, dirs, files in os.walk(chemain):

        for file in files:
            
            # Récupération du chemain d'accès complet de chaque image
            image_file = os.path.join(path, file)
            # Lecture de l'image en cours
            image = cv2.imread(image_file)
            # convertir l'image en cours en image blanc noir
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Initialisation du détecteur sift
            sift = cv2.xfeatures2d.SIFT_create()
            # Caclcul et Extraction des descripteurs de l'image en cours
            keys, descriptor = sift.detectAndCompute(image_gray, None)
            # Stockage des descripteurs extraits de l'image en cours dans un tableau non ordonné
            descripteur_table.extend(descriptor)
            # Création d'un dictionnaire contenant le nom des images avec la liste des descripteurs correspondants
            sift_vectors[file] = descripteur_table
            # Stockage des descripteurs extraits de l'image en cours dans un tableau ordonné par classe
            descripteur.append(descriptor)

    print("******descripteur*******")
    print(descripteur_table[0])
    print("******image descripteur*******")
    print(descripteur[0])
    # Création du sac de mots visuels en utilisant kmeans
    clustering_model = bag_of_word_with_kmeans(n_cluster, descripteur_table)

    # Création de la table des histogrammes des images
    histogram_feature = build_all_images_histogram(clustering_model, descripteur)
    print(len(histogram_feature))
    # Stockage de l'histogramme dans un fichier binaire pour une utilisation ultérieur
    pickle.dump(histogram_feature, open("hist_file", 'wb'))
    print('Fichier histogramme enregistré avec succès')

    # Stockage du modèle kmeans dans un fichier binaire pour une utilisation ultérieur
    pickle.dump(clustering_model, open("model_file", 'wb'))
    print('Modèle kmeans enregistré avec succès')
    # retourner le modèle kmeans et le tableau des histogrammes
    return clustering_model, histogram_feature



In [31]:
# Appel de la fonction indexation et récupération du modèle kmeans et du tableau des histogrammes
kmeans, histograms = indexation(chemain)

******descripteur*******
[ 19.   0.   0.   0.   1.   1.   1.   8. 157.   0.   0.   0.   0.   0.
   0.  54. 157.   0.   0.   0.   0.   0.   0.  93.  68.   0.   0.   0.
   0.   0.   0.  39.  34.   5.   1.   2.   0.   0.   0.   3. 157.   5.
   0.   0.   0.   0.   0.  52. 157.   3.   0.   0.   0.   0.   0.  49.
 107.   5.   0.   0.   0.   0.   0.  10.  36.   3.   0.   1.   1.   1.
   1.   3. 157.  65.   0.   0.   0.   0.   0.   4. 157.  61.   0.   0.
   0.   0.   0.   6.  56.  43.  10.   0.   0.   0.   0.   2.  11.   3.
   0.   1.   0.   0.   0.   3. 157.  80.   0.   0.   0.   0.   0.   2.
 157.  75.   1.   0.   0.   0.   2.   9.  16.  40.  15.   0.   0.   0.
   2.   2.]
******image descripteur*******
[[ 19.   0.   0. ...   0.   2.   2.]
 [ 45. 149.   0. ...   3.   2.   0.]
 [ 24.  85.   5. ...   0.  28. 130.]
 ...
 [  1.   1.   0. ...  12.  13.   7.]
 [ 17.  57.  33. ...   0.   0.   0.]
 [ 15.  17.   7. ...  10.   0.   1.]]
1000
Fichier histogramme enregistré avec succès
Modèle kmeans enr

In [0]:
#histogram_feature = pickle.load( open( "/hist_file", "rb" ) )
#kmeans = pickle.load( open( "/model_file", "rb" ) )

In [0]:
# Tableau contenant le nom des images
image = []
# récupération du nom des images associé aux descripteurs pour effectuer la prédiction avec la fonction recherche
for k, v in sift_vectors.items():
  image.append(k)


In [0]:
# Fonction recherche :  permet de rechercher un nombre N d'images proches d'une image test entrée en paramètres
# Retourne les distance des cluster trouvés ainsi que leur indice et le mapping avec les images de la base.

def recherche(image_path, N):
    #lecture de l'image
    data = cv2.imread(image_path)
    #convertir en blanc noir
    data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
    # extraction des descripteurs sift
    sift = cv2.xfeatures2d.SIFT_create()
    keypoint, descriptor = sift.detectAndCompute(data, None)
    # calcule de l'histogramme de l'image test
    histogram = build_image_histogram(kmeans, descriptor)
    # Recherche des images voisins proche de l'image test en utilisant l'histogramme de toutes les images
    # On fait un entraînement du modèle du plus proche voisin sur l'ensemble des histogrammes des images 
    neighbor = NearestNeighbors(n_neighbors = N)
    neighbor.fit(histograms)
    # récupération des distances à chaque image plus proche trouvé et des indices des clusters correspondants
    dist, result = neighbor.kneighbors([histogram])
    result = np.asarray(result)
    # mapping avec la base d'image et des indices des cluster récupérer pour retrouver les images voisines prédict par le modèle
    image2 = list(map(lambda img: image[int(img)], result[0]))

    # Retourner les distances, les indices des cluster et les images trouvées
    return dist, result , image2


In [34]:
# Chemain de l'image test
test = "/image.orig/39.jpg"
# Test sur une image pour voir comment se présente les résultats
dist, result, imgs = recherche(test, 10)
# Affichage des distances
print(dist)
# Affichage des images
print(imgs)
# Affichage des indices
result

[[0.         0.0004305  0.00044363 0.00044442 0.00044623 0.00045427
  0.00045614 0.00045724 0.00045949 0.00045951]]
['39.jpg', '23.jpg', '257.jpg', '96.jpg', '68.jpg', '64.jpg', '504.jpg', '12.jpg', '47.jpg', '41.jpg']


array([[540,  80, 927, 517, 609, 246, 720, 950, 836, 101]])

In [0]:

# Tableau pour enrégistrer les précisions pour chaque image rechercher avec la fonction recherche
average_for_image = []

# Fonction pour calculer la précision pour chaque image
# Prend comme paramètre le chemain de la base d'image et le nombre N de voisins le plus proche
# Retourne un tableau contenant les précision calculées pour chaque image dans la base
def accuracy(chemain, N):
    # Tableau pour récupérer les précisions pour chaque image
    precision_image = []

    # Boucle pour parcourir la base d'image et rechercher les N voisins les plus proche de chaque image
    for path, dirs, files in os.walk(chemain):
      for file in files:
        # Récupération du nom de l'image sans le format ".jpg" et convertion en int
        k = int(file[:(len(file)-4)])
        # Chemain complet pour accéder à chaque imae
        image_file = os.path.join(path, file)

        # Appel de la fonction recherche pour retrouver les N voisins les plus proche de l'image en cours
        dist, result, img_result = recherche(image_file, N)
        # Tableau contenant les noms des images retrouvées sans le format ".jpg" et convertit en int pour pouvoir
        # bien calculer la moyenne de précision
        tab2 = [int(img[:(len(img)-4)]) for img in img_result]

        # Récupération des prédictions qui sont corrects sur les N images prédict
        # Ici on utilise le nom des images (sans la partie ".jpg") converti en int pour pouvoir faire
        # la correspondance entre le concept de l'image test et le concept des images retournées. On vérifie
        # donc la plage d'entier dans laquelle le nom de l'image se trouve et on récupère dans un tableau les nom des images
        # prédict par le modèle et qui se trouvent dans la même plage d'entier. Ainsi on fait la taille de se tableau divisée par
        # le nombre N de voisins prédict pour calculer la précision pour l'image test.
        if (k <= 99):
            predict = [i for i in tab2 if i <= 99]

        elif (k <= 199):
            predict = [i for i in tab2 if i <= 199 and i >= 100]

        elif (k <= 299):
            predict = [i for i in tab2 if i <= 299 and i >= 200]

        elif (k <= 399):
            predict = [i for i in tab2 if i <= 399 and i >= 300]

        elif (k <= 499):
            predict = [i for i in tab2 if i <= 499 and i >= 400]

        elif (k <= 599):
            predict = [i for i in tab2 if i <= 599 and i >= 500]

        elif (k <= 699):
            predict = [i for i in tab2 if i <= 699 and i >= 600]

        elif (k <= 799):
            predict = [i for i in tab2 if i <= 799 and i >= 700]

        elif (k <= 899):
            predict = [i for i in tab2 if i <= 899 and i >= 800]


        elif (k <= 999):
            predict = [i for i in tab2 if i <= 999 and i >= 900]

        # Calcule de la précision pour l'image en cours
        average = len(predict)/N
        # Ajout de la précision calculée pour l'image en cours dans le tableau des précisions
        precision_image.append(average)


    return precision_image



In [36]:
# Appel de la fonction calculer précision
precision = accuracy(chemain, 10)
precision_somme = 0
# Calcule de la précision moyenne du modèle
for i in precision:
    precision_somme += i

precision_moyenne = precision_somme/len(precision)

# Affichage de la précision moyenne
print(precision_moyenne)

0.47290000000000104


Analyse des résultats Partie 1: 
  - Après avoir calculer la précision moyenne du modèle nous avons obtenu une précision de 48%
  - Par si on prend les résultats par concept, nous remarquons que certains concepts ont
  de bonnes précision des résutats retournés. Par exemple la classe des chevaux (90%), 9/10
  images des chevaux sont retournées pendant la recherche.
  la classe des dinausors aussi (90%), 9/10 images des dinausors sont retournées pendant la recherche.


*Deuxième partie  : Conception d’un moteur de recherche d’images avec structuration des index*
                          
*Méthode utilisée : fréquence inversé tf-idf*


In [0]:

# Importation de la méthode TfidfTransformer de sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import normalize

# Fonction pour calculer le tfidf des images à partir de leur histogramme. Cette fonction
# prends en paramètre les histogrammes des images. Ainsi on aura une structure 
# dans laquelle à chaque mot visuel de l'histogramme va correspondre
# une liste d'images dans laquelle ce mot viuel apparait ainsi que sa fréquence
# d'apparition.
def tfidf(bow):
	# td-idf initialisation
  transformer = TfidfTransformer(smooth_idf=True)
  t = transformer.fit_transform(bow).toarray()
        
  # normaliser par la norme euclidienne (L2) avant de retourner le modèle tf-idf 
  t = normalize(t, norm='l2', axis=1)
    
  return t

# Appel de la méthode tfidf sur la table des histogrammes des images
histograms = np.vstack(histograms)
t = tfidf(histograms)

# Enrégistrement du tfidf de toutes les images avec pickle
pickle.dump(t, open("tfidf.pkl", "wb"))

In [0]:
# Fonction de recherche d'une image à partir de la fréquence d'apparution de ses
# mots visuels. On récupère les mots visuels de l'image et on recherche les images de la base
# contenant l'un des mots visuels de l'image test.
def recherche_with_tfidf(image_path, N):
    #lecture de l'image
    data = cv2.imread(image_path)
    #convertir en blanc noir
    data = cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
    # extraction des descripteurs sift
    sift = cv2.xfeatures2d.SIFT_create()
    keypoint, descriptor = sift.detectAndCompute(data, None)
    # calcule de l'histogramme de l'image test
    histogram = build_image_histogram(kmeans, descriptor)
    # calcule du tfidf à partir l'histogramme de l'image test
    image_test = tfidf(np.vstack(histogram))
    image_test_flatten = np.array(image_test).flatten()
    # Récupération du tfidf de toutes les images
    allimages_tfidf = t
    # Recherche des images voisins proche de l'image test en utilisant le tfidf de toutes les images
    neighbor = NearestNeighbors(n_neighbors = N)
    neighbor.fit(allimages_tfidf)
    # récupération des distances à chaque image plus proche trouvé
    dist, result = neighbor.kneighbors([image_test_flatten])
    result = np.asarray(result)
    # mapping avec la base d'image et des indices des cluster récupérer pour retrouver les images voisines prédict par le modèle
    image2 = list(map(lambda img: image[int(img)], result[0]))

    # Retourner les images trouvées
    return image2

In [0]:
# Tableau pour enrégistrer les précisions pour chaque image rechercher avec la fonction recherche
average_for_image = []

# Fonction pour calculer la précision pour chaque image
# Prend comme paramètre le chemain de la base d'image et le nombre N de voisins le plus proche
# Retourne un tableau contenant les précision calculées pour chaque image dans la base
def accuracy2(chemain, N):
    # Tableau pour récupérer les précisions pour chaque image
    precision_image = []

    # Boucle pour parcourir la base d'image et rechercher les N voisins les plus proche de chaque image
    for path, dirs, files in os.walk(chemain):
      for file in files:
        # Récupération du nom de l'image sans le format ".jpg" et convertion en int
        k = int(file[:(len(file)-4)])
        # Chemain complet pour accéder à chaque imae
        image_file = os.path.join(path, file)

        # Appel de la fonction recherche pour retrouver les N voisins les plus proche de l'image en cours
        img_result = recherche_with_tfidf(image_file, N)
        # Tableau contenant les noms des images retrouvées sans le format ".jpg" et convertit en int pour pouvoir
        # bien calculer la moyenne de précision
        tab2 = [int(img[:(len(img)-4)]) for img in img_result]

        # Récupération des prédictions qui sont corrects sur les N images prédict
        # Ici on utilise le nom des images (sans la partie ".jpg") converti en int pour pouvoir faire
        # la correspondance entre le concept de l'image test et le concept des images retournées. On vérifie
        # donc la plage d'entier dans laquelle le nom de l'image se trouve et on récupère dans un tableau les nom des images
        # prédict par le modèle et qui se trouvent dans la même plage d'entier. Ainsi on fait la taille de se tableau divisée par
        # le nombre N de voisins prédict pour calculer la précision pour l'image test.
        if (k <= 99):
            predict = [i for i in tab2 if i <= 99]

        elif (k <= 199):
            predict = [i for i in tab2 if i <= 199 and i >= 100]

        elif (k <= 299):
            predict = [i for i in tab2 if i <= 299 and i >= 200]

        elif (k <= 399):
            predict = [i for i in tab2 if i <= 399 and i >= 300]

        elif (k <= 499):
            predict = [i for i in tab2 if i <= 499 and i >= 400]

        elif (k <= 599):
            predict = [i for i in tab2 if i <= 599 and i >= 500]

        elif (k <= 699):
            predict = [i for i in tab2 if i <= 699 and i >= 600]

        elif (k <= 799):
            predict = [i for i in tab2 if i <= 799 and i >= 700]

        elif (k <= 899):
            predict = [i for i in tab2 if i <= 899 and i >= 800]


        elif (k <= 999):
            predict = [i for i in tab2 if i <= 999 and i >= 900]

        # Calcule de la précision pour l'image en cours
        average = len(predict)/N
        # Ajout de la précision calculée pour l'image en cours dans le tableau des précisions
        precision_image.append(average)


    return precision_image

In [28]:
# Appel de la fonction calculer précision
precision = accuracy2(chemain, 10)
precision_somme = 0
# Calcule de la précision moyenne du modèle
for i in precision:
    precision_somme += i

precision_moyenne = precision_somme/len(precision)

# Affichage de la précision moyenne
print(precision_moyenne)

0.18049999999999894


Analyse des résultats Partie 2 : 
  - Après avoir calculer la précision moyenne du modèle en utilisant le tf-idf nous avons obtenu une précision myenne de 18%. Il y a donc une baisse de la précision par rapport à la première méthode. Cette baisse peut s'expliquer par le fait que on fait la recherche par rapport à la présence d'un des mots visuels de l'image recherchée dans la base d'images. Le résultat retourné peut donc contenir un mot visuel de l'image recherchée mais ne va pas correspondre réellement au même concept d'image.
  - Nous remarquons aussi que la recherche est beaucoup plus rapide que par rapport à la première partie
  